## Q1. Downloading the data


We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [1]:
import os
import pandas as pd
import numpy as np


In [2]:
df = pd.read_parquet(os.path.join("data", "yellow_tripdata_2023-01.parquet"))

In [3]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [4]:
len(df.columns)

19

## Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?


In [5]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [6]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [7]:
df['duration'] = ( df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime'] )

In [8]:
df['duration'] = df['duration'].dt.total_seconds() / 60

In [9]:
df['duration'].head()

0     8.433333
1     6.316667
2    12.750000
3     9.616667
4    10.833333
Name: duration, dtype: float64

In [10]:
duration_std_dev = df['duration'].std()
duration_std_dev

42.59435124195458

## Q3. Dropping outliers


Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [11]:
df_wo_outliers = df[ (df['duration'] >=1.0) & (df['duration'] <= 60.0) ]

In [12]:
fraction_left = len(df_wo_outliers) / len(df)
fraction_left

0.9812202822125979

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
* Fit a dictionary vectorizer
* Get a feature matrix from it
* What's the dimensionality of this matrix (number of columns)?

In [13]:
from sklearn.feature_extraction import DictVectorizer


In [14]:
dff = df_wo_outliers.copy()

In [15]:
dff.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object

In [16]:
data = dff[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

In [17]:
data

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'},
 {'PULocationID': '161', 'DOLocationID': '137'},
 {'PULocationID': '239', 'DOLocationID': '143'},
 {'PULocationID': '142', 'DOLocationID': '200'},
 {'PULocationID': '164', 'DOLocationID': '236'},
 {'PULocationID': '141', 'DOLocationID': '107'},
 {'PULocationID': '234', 'DOLocationID': '68'},
 {'PULocationID': '79', 'DOLocationID': '264'},
 {'PULocationID': '164', 'DOLocationID': '143'},
 {'PULocationID': '138', 'DOLocationID': '33'},
 {'PULocationID': '33', 'DOLocationID': '61'},
 {'PULocationID': '79', 'DOLocationID': '186'},
 {'PULocationID': '90', 'DOLocationID': '48'},
 {'PULocationID': '113', 'DOLocationID': '255'},
 {'PULocationID': '237', 'DOLocationID': '239'},
 {'PULocationID': '143', 'DOLocationID': '229'},
 {'PULocationID': '137', 'DOLocat

In [18]:
vectorizer = DictVectorizer()
vectorizer.fit(data)

DictVectorizer()

In [19]:
feature_matrix = vectorizer.fit_transform(data)


In [20]:
feature_matrix.shape

(3009173, 515)

In [21]:
num_columns = feature_matrix.shape[1]
num_columns

515

## Data Preparation

In [22]:
def data_prep(df: pd.DataFrame) -> tuple:
    df['duration'] = ( df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime'] )
    df['duration'] = df['duration'].dt.total_seconds() / 60
    
    dff = df[ (df['duration'] >=1.0) & (df['duration'] <= 60.0) ]
    return dff


## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [24]:
tf1 = pd.read_parquet(os.path.join("data", "yellow_tripdata_2023-01.parquet"))

In [25]:
df1 = data_prep(tf1)

In [26]:
X1 = vectorizer.fit_transform( df1[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records') )

In [27]:
y1 = df1['duration']

In [28]:
# X1_train, X1_test, y1_train, y1_test  = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [29]:
model = LinearRegression()
model.fit(X1, y1)


LinearRegression()

In [30]:
y1_pred = model.predict(X1)


In [31]:
mse = mean_squared_error(y1, y1_pred)
np.sqrt(mse)


7.6492620683865304

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [32]:
tf2 = pd.read_parquet(os.path.join("data", "yellow_tripdata_2023-02.parquet"))

In [33]:
df2 = data_prep(tf2)

In [34]:
data2 = df2[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

In [35]:
X2 = vectorizer.transform(data2)


In [36]:
y2 = df2['duration']
y2

0           1.683333
3          32.083333
4          13.300000
5          14.633333
6          27.950000
             ...    
2913950    19.000000
2913951    11.133333
2913952    14.000000
2913953     7.000000
2913954     9.800000
Name: duration, Length: 2855951, dtype: float64

In [37]:
y2_pred = model.predict(X2)

In [38]:
mse = mean_squared_error(y2, y2_pred)
np.sqrt(mse)


7.81181571625934